In [83]:
import sys
import pandas as pd
import numpy as np
import random
from collections import Counter
import scipy as sp
from scipy import stats, optimize, interpolate
import statsmodels.api as sm
import statsmodels as sm
random.seed(0)

In [22]:
main_dir ="/u/home/b/briscoel/project-halperin/MicroBatch/data/"
script_folder= "/u/home/b/briscoel/project-halperin/MicroBatch/RevisionSequence/"
local = True

sys.argv = [ "Thomasr_complete_otu","rel",""]

if local:
    main_dir = "/Users/leahbriscoe/Documents/MicroBatch/microbatch_vc/data/"
    script_folder = "/Users/leahbriscoe/Documents/MicroBatch/microbatch_vc/RevisionSequence/"

folder = sys.argv[0] # "AGPr_max_k5" #"AGPr_complete_otu" 
trans = sys.argv[1] #"rel"
correction= sys.argv[2] #"rel"
data_dir = main_dir + folder + "/"
d1 = 'FengQ_2015'
d2 = 'ZellerG_2014'


In [13]:
metadata_table = pd.read_csv(data_dir + "metadata.txt", delimiter = "\t",header=0)

if correction == "":
    feature_table = pd.read_csv(data_dir  + "feature_table_" + trans + ".txt" ,delimiter = "\t",header=0)
else:
    feature_table = pd.read_csv(data_dir  +"feature_table_" + trans + "_" + correction + ".txt" ,delimiter = "\t",header=0)

feature_table_presence = feature_table  > 0     

In [50]:
all_control_samples = list(metadata_table[metadata_table['bin_crc_normal'] == 0].index)
all_case_samples = list(metadata_table[metadata_table['bin_crc_normal'] == 1].index)
control_presence_proportion = (feature_table_presence[all_control_samples].sum(axis =1))/len(all_control_samples)
case_presence_proportion = (feature_table_presence[all_case_samples].sum(axis =1))/len(all_case_samples)


In [55]:

## FILTER FOR 1/3 in at least case OR control : not using for now
prevalence_filter = np.array(control_presence_proportion > 0.33) | np.array(case_presence_proportion > 0.33)

123

In [29]:
control_metadata1 = metadata_table[(metadata_table['bin_crc_normal'] == 0 ) & (metadata_table['dataset_name'] ==  d1) ]
case_metadata1 = metadata_table[(metadata_table['bin_crc_normal'] == 1) & (metadata_table['dataset_name'] ==  d1)]

control_metadata2 = metadata_table[(metadata_table['bin_crc_normal'] == 0 ) & (metadata_table['dataset_name'] ==  d2) ]
case_metadata2 = metadata_table[(metadata_table['bin_crc_normal'] == 1) & (metadata_table['dataset_name'] ==  d2)]

control_samples_d1 = list(control_metadata1.index)
case_samples_d1 = list(case_metadata1.index)

control_samples_d2 = list(control_metadata2.index)
case_samples_d2 = list(case_metadata2.index)

min_sample_size = min([len(control_samples_d1),len(case_samples_d1),len(control_samples_d2),len(case_samples_d2)])

In [30]:
[len(control_samples_d1),len(case_samples_d1),len(control_samples_d2),len(case_samples_d2)]

[61, 46, 66, 91]

In [32]:
control_samples_d1 = random.sample(control_samples_d1,min_sample_size)
case_samples_d1 = random.sample(case_samples_d1,min_sample_size)
control_samples_d2 = random.sample(control_samples_d2,min_sample_size)
case_samples_d2 = random.sample(case_samples_d2,min_sample_size)

In [87]:
len(pvals_fdr)

167

In [88]:
replicate_titrate_rel_names = {}
replicate_titrate_rel_fdr = {}



titrate_rel_names = {}
titrate_rel_fdr = {}

fractions = [0, .25 , .50, 1] # fraction of controls replaced
#for n in range(5):
for f in fractions:
    pvals = []
    correction = "rel"
    case_samples = case_samples_d1
    control_samples = random.sample(control_samples_d1,int((1-f) * min_sample_size)) + random.sample(control_samples_d2, int(f * min_sample_size))
    print(len(control_samples))
    feature_table_titrate_control = feature_table[control_samples ]
    feature_table_titrate_case = feature_table[case_samples ]
    print("dim feature table titrate")
    print(feature_table_titrate_control.shape)
    print(feature_table_titrate_case.shape)
    # for each taxa
    for tax in range(feature_table_titrate_case.shape[0]):
        if feature_table_titrate_control.iloc[tax,:].astype(bool).sum()/float(feature_table_titrate_control.shape[1]) >= 1/3.0 or feature_table_titrate_case.iloc[tax,:].astype(bool).sum()/float(feature_table_titrate_case.shape[1]) >= 1/3.0:
            pvals.append(sp.stats.ranksums(feature_table_titrate_control.iloc[tax,:],feature_table_titrate_case.iloc[tax,:])[1])
        else:
            pvals.append(1)
        
    pvals = np.array(pvals)
    pvals_fdr = sm.sandbox.stats.multicomp.multipletests(np.array(pvals),alpha=0.05,method='fdr_bh')[1]
    
    names = []
    fdr = []

    for j in range(len(pvals_fdr)):
        if pvals_fdr[j] <= 0.05:
            names.append(feature_table_titrate_case.index[j])
            fdr.append(pvals_fdr[j])
            
    titrate_rel_names[f] = names
    titrate_rel_fdr[f] = fdr
n = 1
replicate_titrate_rel_names[n] = titrate_rel_names
replicate_titrate_rel_fdr[n] = titrate_rel_fdr

    

46
dim feature table titrate
(830, 46)
(830, 46)
45
dim feature table titrate
(830, 45)
(830, 46)
46
dim feature table titrate
(830, 46)
(830, 46)
46
dim feature table titrate
(830, 46)
(830, 46)


In [91]:
 titrate_rel_names

{0: ['s__Alistipes_finegoldii',
  's__Porphyromonas_asaccharolytica',
  's__Dialister_invisus',
  's__Prevotella_copri',
  's__Ruminococcaceae_bacterium_D16',
  's__Parvimonas_unclassified',
  's__Gemella_morbillorum',
  's__Fusobacterium_nucleatum',
  's__Dasheen_mosaic_virus'],
 0.25: ['s__Dialister_invisus', 's__Prevotella_copri'],
 0.5: ['s__Eubacterium_biforme', 's__Dialister_invisus'],
 1: ['s__Streptococcus_australis',
  's__Bifidobacterium_adolescentis',
  's__Parasutterella_excrementihominis',
  's__Eubacterium_eligens',
  's__Eubacterium_biforme',
  's__Collinsella_aerofaciens',
  's__Odoribacter_splanchnicus',
  's__Fusobacterium_nucleatum',
  's__Dasheen_mosaic_virus']}